In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [13]:
!pip install deepod

    jupyter-core!=~5.0,>=4.12
                ^


In [14]:
#!pip install torcheval
!pip install seaborn

    jupyter-core!=~5.0,>=4.12
                ^


In [15]:
!pip install torcheval

    jupyter-core!=~5.0,>=4.12
                ^


In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import os
from pathlib import Path

path = Path('../data')#Path("/content/drive/MyDrive/이어드림/chohj_skin/data")
train_img_path = path /'train-image/image'

## Data Check

In [19]:
train_csv = pd.read_csv(path / 'train-metadata.csv', low_memory=False)
print(f'train_csv shape : {train_csv.shape}')

train_csv shape : (401059, 55)


In [24]:
!pip install colorama==0.4.6
!pip install timm==0.9.11

    jupyter-core!=~5.0,>=4.12
                ^
    jupyter-core!=~5.0,>=4.12
                ^


In [25]:
from PIL import Image
import os
import gc
import cv2
import math
import copy
import time
import random
from glob import glob

# torch importsa
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
import torcheval
from torcheval.metrics.functional import binary_auroc

import timm

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# 시드 고정

SEED = 12

def set_seed(SEED):
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(SEED)

set_seed(SEED)


In [26]:
# 메타데이터 헤드 확인
train_csv.head()

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


In [27]:
# 결측치 확인
train_csv.isna().sum()

isic_id                              0
target                               0
patient_id                           0
age_approx                        2798
sex                              11517
anatom_site_general               5756
clin_size_long_diam_mm               0
image_type                           0
tbp_tile_type                        0
tbp_lv_A                             0
tbp_lv_Aext                          0
tbp_lv_B                             0
tbp_lv_Bext                          0
tbp_lv_C                             0
tbp_lv_Cext                          0
tbp_lv_H                             0
tbp_lv_Hext                          0
tbp_lv_L                             0
tbp_lv_Lext                          0
tbp_lv_areaMM2                       0
tbp_lv_area_perim_ratio              0
tbp_lv_color_std_mean                0
tbp_lv_deltaA                        0
tbp_lv_deltaB                        0
tbp_lv_deltaL                        0
tbp_lv_deltaLB           

In [28]:
def get_img_path(image_id):
    return f"{train_img_path}/{image_id}.jpg"


# Count the Images
train_img_path = path / 'train-image/image'
train_images = glob(str(train_img_path) + '/*')

df_positive = train_csv[train_csv["target"] == 1].reset_index(drop=True)
df_negative = train_csv[train_csv["target"] == 0].reset_index(drop=True)

print(f"Total num of train_img set: {len(train_images)}")
print(f'Train metadata shape : {train_csv.shape}')
print(f'sum of the targets: {train_csv.target.sum()}')
print(f'{train_csv["patient_id"].unique().shape}')
print(df_positive.shape)
print(df_negative.shape)

# 비율 맞추기(baseline 참고시 1: 20으로 했는데 좀 아이디어 있으면 고치시면 됩니다!)
# 데이터 불균형 존재
df = pd.concat([df_positive, df_negative.iloc[:df_positive.shape[0]*20, :]])
print("filtered>", df.shape, df.target.sum(), df["patient_id"].unique().shape)
df['file_path'] = df['isic_id'].apply(get_img_path)
df = df[ df["file_path"].isin(train_images) ].reset_index(drop=True)
df

Total num of train_img set: 401059
Train metadata shape : (401059, 55)
sum of the targets: 393
(1042,)
(393, 55)
(400666, 55)
filtered> (8253, 55) 393 (950,)


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence,file_path
0,ISIC_0082829,1,IP_3249371,80.0,female,anterior torso,9.27,TBP tile: close-up,3D: XP,18.093368,...,Malignant::Malignant epidermal proliferations:...,Malignant,Malignant epidermal proliferations,Squamous cell carcinoma in situ,NaN,NaN,NaN,NaN,72.211683,../data/train-image/image/ISIC_0082829.jpg
1,ISIC_0096034,1,IP_6723298,75.0,male,head/neck,3.88,TBP tile: close-up,3D: white,26.187010,...,Malignant::Malignant adnexal epithelial prolif...,Malignant,Malignant adnexal epithelial proliferations - ...,Basal cell carcinoma,"Basal cell carcinoma, Nodular",NaN,NaN,NaN,94.559590,../data/train-image/image/ISIC_0096034.jpg
2,ISIC_0104229,1,IP_9057861,80.0,male,anterior torso,6.55,TBP tile: close-up,3D: white,33.480140,...,Malignant::Malignant adnexal epithelial prolif...,Malignant,Malignant adnexal epithelial proliferations - ...,Basal cell carcinoma,"Basal cell carcinoma, Nodular",NaN,NaN,NaN,66.196170,../data/train-image/image/ISIC_0104229.jpg
3,ISIC_0119495,1,IP_6856511,60.0,male,upper extremity,5.27,TBP tile: close-up,3D: white,25.872050,...,Malignant::Malignant epidermal proliferations:...,Malignant,Malignant epidermal proliferations,"Squamous cell carcinoma, Invasive",NaN,NaN,NaN,NaN,99.991270,../data/train-image/image/ISIC_0119495.jpg
4,ISIC_0157834,1,IP_3927284,55.0,male,posterior torso,5.29,TBP tile: close-up,3D: white,30.966662,...,Malignant::Malignant adnexal epithelial prolif...,Malignant,Malignant adnexal epithelial proliferations - ...,Basal cell carcinoma,"Basal cell carcinoma, Superficial",NaN,NaN,NaN,99.960846,../data/train-image/image/ISIC_0157834.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8248,ISIC_0267522,0,IP_9577633,50.0,male,lower extremity,4.94,TBP tile: close-up,3D: XP,21.738880,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,96.325510,../data/train-image/image/ISIC_0267522.jpg
8249,ISIC_0267560,0,IP_7746572,80.0,male,anterior torso,9.32,TBP tile: close-up,3D: white,18.481750,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.022320,../data/train-image/image/ISIC_0267560.jpg
8250,ISIC_0267568,0,IP_0379091,60.0,male,anterior torso,3.45,TBP tile: close-up,3D: white,13.901465,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,72.961688,../data/train-image/image/ISIC_0267568.jpg
8251,ISIC_0267594,0,IP_1433033,70.0,NaN,lower extremity,3.00,TBP tile: close-up,3D: XP,17.986882,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.899274,../data/train-image/image/ISIC_0267594.jpg


In [29]:
# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold

sgkf = StratifiedGroupKFold(n_splits=5)

for fold, ( _, val_) in enumerate(sgkf.split(df, df.target,df.patient_id)):
      df.loc[val_ , "kfold"] = int(fold)

In [31]:
class SkinDataSet(Dataset):
    def __init__(self, df, transforms=None):
        self.df_positive = df[df['target'] == 1].reset_index()
        self.df_negative = df[df['target'] == 0].reset_index()
        self.file_names_positive = self.df_positive['file_path'].values
        self.file_names_negative = self.df_negative['file_path'].values
        self.targets_positive = self.df_positive['target'].values
        self.targets_negative = self.df_negative['target'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.df_positive) * 2

    def __getitem__(self, index):
        if random.random() >= 0.5:
            df = self.df_positive
            file_names = self.file_names_positive
            targets = self.targets_positive
        else:
            df = self.df_negative
            file_names = self.file_names_negative
            targets = self.targets_negative

        index = index % df.shape[0]

        img_path = file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = targets[index]

        if self.transforms:
            img = self.transforms(image=img)['image']

        return {'image': img, 'target': target}


class ISICDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.targets = df['target'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.targets[index]

        if self.transforms:
            img = self.transforms(image=img)["image"]

        return {
            'image': img,
            'target': target
        }

## Augmentation (증강시 증강하세요!)

In [32]:
from deepod.models.tabular import DeepSAD

In [33]:
clf = DeepSAD()

In [34]:
train_y=train_csv["target"]

In [35]:
train_x=train_csv.drop("target",axis=1)

In [36]:
train_x

,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,16.261975,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,25.364740,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,17.128170,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,12.164757,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,20.057470,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,IP_1140263,70.0,male,anterior torso,6.80,TBP tile: close-up,3D: XP,22.574335,14.944666,...,IL_9520694,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999988
401055,ISIC_9999951,IP_5678181,60.0,male,posterior torso,3.11,TBP tile: close-up,3D: white,19.977640,16.026870,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999820
401056,ISIC_9999960,IP_0076153,65.0,female,anterior torso,2.05,TBP tile: close-up,3D: XP,17.332567,12.364397,...,IL_9852274,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999416
401057,ISIC_9999964,IP_5231513,30.0,female,anterior torso,2.80,TBP tile: close-up,3D: XP,22.288570,9.564721,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000


In [37]:
dropcolumns=[]
for i in train_x.columns:
  if (train_x[i].dtype=='object'):
    dropcolumns.append(i)

In [38]:
dropcolumns

['isic_id',
 'patient_id',
 'sex',
 'anatom_site_general',
 'image_type',
 'tbp_tile_type',
 'tbp_lv_location',
 'tbp_lv_location_simple',
 'attribution',
 'copyright_license',
 'lesion_id',
 'iddx_full',
 'iddx_1',
 'iddx_2',
 'iddx_3',
 'iddx_4',
 'iddx_5',
 'mel_mitotic_index']

In [39]:
train_x=train_x.drop(columns =dropcolumns)

In [40]:
train_y.to_numpy()

array([0, 0, 0, ..., 0, 0, 0])

In [41]:
train_x.to_numpy()

array([[  60.        ,    3.04      ,   20.2444223 , ...,  -42.427948  ,
                  nan,   97.51728177],
       [  60.        ,    1.1       ,   31.71257   , ...,   57.1745    ,
                  nan,    3.141455  ],
       [  60.        ,    3.4       ,   22.57583   , ...,  232.9089    ,
                  nan,   99.80404   ],
       ...,
       [  65.        ,    2.05      ,   17.33256672, ..., -106.83374023,
                  nan,   99.99941587],
       [  30.        ,    2.8       ,   22.28857   , ...,  -76.98212   ,
                  nan,  100.        ],
       [  50.        ,    3.3       ,   16.7929    , ...,  107.7106    ,
                  nan,   99.99996   ]])

In [42]:
clf.fit(train_x.to_numpy(), train_y.to_numpy())

Start Training...
ensemble size: 1
training data counter: Counter({0: 400666, -1: 393})
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=36, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: nan, time: 37.9s
epoch 10, training loss: nan, time: 35.4s
epoch 20, training loss: nan, time: 34.9s
epoch 30, training loss: nan, time: 35.3s
epoch 40, training loss: nan, time: 13.7s
epoch 50, training loss: nan, time: 35.0s
epoch 60, training loss: nan, time: 34.2s
epoch 70, training loss: nan, time: 35.3s
epoch 80, training loss: nan, time: 34.9s
epoch 90, training loss: nan, time: 35.2s
epoch100, training loss: nan, time: 29.5s
Start Inference on the training data..

testing: 100%|██████████| 6267/6267 [00:03<00:00, 1827.95it/s]


In [27]:
scores = clf.predict(train_x.to_numpy())



testing: 100%|██████████| 39/39 [00:00<00:00, 683.34it/s]


In [28]:
scores

array([0, 0, 0, ..., 0, 0, 0])

In [29]:
# evaluation of tabular anomaly detection
from deepod.metrics import tabular_metrics
auc, ap, f1 = tabular_metrics(train_y.to_numpy(), scores)

In [31]:
print (auc,ap,f1)

0.5 0.001637331150225133 0.0032693093583980384


In [1]:
pred

NameError: name 'pred' is not defined

################################################################################################################################################################

In [ ]:
!pip install -r '/content/drive/MyDrive/이어드림/chohj_skin/Deep-SAD-PyTorch-master/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.2/37.2 MB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 42.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 62.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!git clone https://github.com/lukasruff/Deep-SAD-PyTorch.git

Cloning into 'Deep-SAD-PyTorch'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 109 (delta 4), reused 6 (delta 2), pack-reused 100 (from 1)
Receiving objects: 100% (109/109), 674.70 KiB | 4.35 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [ ]:
!pip install virtualenv
%cd <path-to-Deep-SAD-PyTorch-directory>
!virtualenv myenv
!source myenv/bin/activate

[Errno 2] No such file or directory: '<path-to-Deep-SAD-PyTorch-directory>'
/content
created virtual environment CPython3.10.12.final.0-64 in 313ms
  creator CPython3Posix(dest=/content/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.1, setuptools==70.1.0, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
%cd /content/Deep-SAD-PyTorch

/content/Deep-SAD-PyTorch


In [ ]:
!pip install base

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for base: filename=base-0.0.0-py3-none-any.whl size=896 sha256=0c285b9003872aa442ec9f8a68085bb2e77d090a523509e9c46a1f24a2bdfa2e
  Stored in directory: /root/.cache/pip/wheels/74/c3/dd/dc916dc3cf0fb35b6a9d9397baa34f779576bd0357ca6ede1f
Successfully built base


In [ ]:
import json
import torch

from base.base_dataset import BaseADDataset
from networks.main import build_network, build_autoencoder
from optim.DeepSAD_trainer import DeepSADTrainer
from optim.ae_trainer import AETrainer


class DeepSAD(object):
    """A class for the Deep SAD method.

    Attributes:
        eta: Deep SAD hyperparameter eta (must be 0 < eta).
        c: Hypersphere center c.
        net_name: A string indicating the name of the neural network to use.
        net: The neural network phi.
        trainer: DeepSADTrainer to train a Deep SAD model.
        optimizer_name: A string indicating the optimizer to use for training the Deep SAD network.
        ae_net: The autoencoder network corresponding to phi for network weights pretraining.
        ae_trainer: AETrainer to train an autoencoder in pretraining.
        ae_optimizer_name: A string indicating the optimizer to use for pretraining the autoencoder.
        results: A dictionary to save the results.
        ae_results: A dictionary to save the autoencoder results.
    """

    def __init__(self, eta: float = 1.0):
        """Inits DeepSAD with hyperparameter eta."""

        self.eta = eta
        self.c = None  # hypersphere center c

        self.net_name = None
        self.net = None  # neural network phi

        self.trainer = None
        self.optimizer_name = None

        self.ae_net = None  # autoencoder network for pretraining
        self.ae_trainer = None
        self.ae_optimizer_name = None

        self.results = {
            'train_time': None,
            'test_auc': None,
            'test_time': None,
            'test_scores': None,
        }

        self.ae_results = {
            'train_time': None,
            'test_auc': None,
            'test_time': None
        }

    def set_network(self, net_name):
        """Builds the neural network phi."""
        self.net_name = net_name
        self.net = build_network(net_name)

    def train(self, dataset: BaseADDataset, optimizer_name: str = 'adam', lr: float = 0.001, n_epochs: int = 50,
              lr_milestones: tuple = (), batch_size: int = 128, weight_decay: float = 1e-6, device: str = 'cuda',
              n_jobs_dataloader: int = 0):
        """Trains the Deep SAD model on the training data."""

        self.optimizer_name = optimizer_name
        self.trainer = DeepSADTrainer(self.c, self.eta, optimizer_name=optimizer_name, lr=lr, n_epochs=n_epochs,
                                      lr_milestones=lr_milestones, batch_size=batch_size, weight_decay=weight_decay,
                                      device=device, n_jobs_dataloader=n_jobs_dataloader)
        # Get the model
        self.net = self.trainer.train(dataset, self.net)
        self.results['train_time'] = self.trainer.train_time
        self.c = self.trainer.c.cpu().data.numpy().tolist()  # get as list

    def test(self, dataset: BaseADDataset, device: str = 'cuda', n_jobs_dataloader: int = 0):
        """Tests the Deep SAD model on the test data."""

        if self.trainer is None:
            self.trainer = DeepSADTrainer(self.c, self.eta, device=device, n_jobs_dataloader=n_jobs_dataloader)

        self.trainer.test(dataset, self.net)

        # Get results
        self.results['test_auc'] = self.trainer.test_auc
        self.results['test_time'] = self.trainer.test_time
        self.results['test_scores'] = self.trainer.test_scores

    def pretrain(self, dataset: BaseADDataset, optimizer_name: str = 'adam', lr: float = 0.001, n_epochs: int = 100,
                 lr_milestones: tuple = (), batch_size: int = 128, weight_decay: float = 1e-6, device: str = 'cuda',
                 n_jobs_dataloader: int = 0):
        """Pretrains the weights for the Deep SAD network phi via autoencoder."""

        # Set autoencoder network
        self.ae_net = build_autoencoder(self.net_name)

        # Train
        self.ae_optimizer_name = optimizer_name
        self.ae_trainer = AETrainer(optimizer_name, lr=lr, n_epochs=n_epochs, lr_milestones=lr_milestones,
                                    batch_size=batch_size, weight_decay=weight_decay, device=device,
                                    n_jobs_dataloader=n_jobs_dataloader)
        self.ae_net = self.ae_trainer.train(dataset, self.ae_net)

        # Get train results
        self.ae_results['train_time'] = self.ae_trainer.train_time

        # Test
        self.ae_trainer.test(dataset, self.ae_net)

        # Get test results
        self.ae_results['test_auc'] = self.ae_trainer.test_auc
        self.ae_results['test_time'] = self.ae_trainer.test_time

        # Initialize Deep SAD network weights from pre-trained encoder
        self.init_network_weights_from_pretraining()

    def init_network_weights_from_pretraining(self):
        """Initialize the Deep SAD network weights from the encoder weights of the pretraining autoencoder."""

        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict()

        # Filter out decoder network keys
        ae_net_dict = {k: v for k, v in ae_net_dict.items() if k in net_dict}
        # Overwrite values in the existing state_dict
        net_dict.update(ae_net_dict)
        # Load the new state_dict
        self.net.load_state_dict(net_dict)

    def save_model(self, export_model, save_ae=True):
        """Save Deep SAD model to export_model."""

        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict() if save_ae else None

        torch.save({'c': self.c,
                    'net_dict': net_dict,
                    'ae_net_dict': ae_net_dict}, export_model)

    def load_model(self, model_path, load_ae=False, map_location='cpu'):
        """Load Deep SAD model from model_path."""

        model_dict = torch.load(model_path, map_location=map_location)

        self.c = model_dict['c']
        self.net.load_state_dict(model_dict['net_dict'])

        # load autoencoder parameters if specified
        if load_ae:
            if self.ae_net is None:
                self.ae_net = build_autoencoder(self.net_name)
            self.ae_net.load_state_dict(model_dict['ae_net_dict'])

    def save_results(self, export_json):
        """Save results dict to a JSON-file."""
        with open(export_json, 'w') as fp:
            json.dump(self.results, fp)

    def save_ae_results(self, export_json):
        """Save autoencoder results dict to a JSON-file."""
        with open(export_json, 'w') as fp:
            json.dump(self.ae_results, fp)

ModuleNotFoundError: No module named 'base'

In [ ]:
# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

data_transforms = {
    "train": A.Compose([
        A.Resize(384, 384),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Downscale(p=0.25),
        A.ShiftScaleRotate(shift_limit=0.1,
                           scale_limit=0.15,
                           rotate_limit=60,
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2,
                sat_shift_limit=0.2,
                val_shift_limit=0.2,
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1),
                contrast_limit=(-0.1, 0.1),
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2()], p=1.),

    "valid": A.Compose([
        A.Resize(384, 384),
        A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

## GemPooling

In [ ]:
class GemPooling(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)

    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

## Modeling

In [ ]:
# ### Modeling 시도할때 timm이 변형하기 좋음
# ## 우선은 베이스라인대로 timm을 애용합시다.

# import timm

# class SkinModel(nn.Module):
#     def __init__(self, model_name, num_classes=1, pretrained=True, checkpoint_path=None):
#         super(SkinModel, self).__init__()
#         self.model = timm.create_model(model_name, pretrained=pretrained, checkpoint_path=checkpoint_path)

#         in_features = self.model.classifier.in_features
#         self.model.classifier = nn.Identity()
#         self.model.global_pool = nn.Identity()
#         self.pooling = GeM()
#         self.linear = nn.Linear(in_features, num_classes)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, images):
#         features = self.model(images)
#         pooled_features = self.pooling(features).flatten(1)
#         output = self.sigmoid(self.linear(pooled_features))
#         return output


# model = SkinModel(model_name, checkpoint_path=checkpoint_path)
# model.to('cuda')

In [ ]:
class SkinModel(nn.Module):
    def __init__(self, num_classes=1):
        super(SkinModel, self).__init__()
        self.model = torchvision.models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V2')
        self.model.requires_grad_ = False
        self.model.fc.requires_grad_ = True
        self.model.fc = nn.Linear(2048, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, images):
        features = self.model(images).flatten(1)
        output = self.sigmoid(features)
        return output

model = SkinModel()
model.to('cuda')

SkinModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [ ]:
def criterion(outputs, targets):
    return nn.BCELoss()(outputs, targets)

In [ ]:
def model_train(model, optimizer, scheduler, dataloader, epoch):
    model.train()

    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to('cuda', dtype=torch.float)
        targets = data['target'].to('cuda', dtype=torch.float)

        batch_size = images.size(0)

        outputs = model(images).squeeze()
        loss = criterion(outputs, targets)

        # gradient acculmuation 사용 시 코드 추가할 것
        # loss = loss / grad_accul

        loss.backward()

        optimizer.step()

        if scheduler is not None:
            scheduler.step()

        auroc = binary_auroc(input=outputs.squeeze(), target=targets).item()

        running_loss += (loss.item() * batch_size)
        running_auroc += (auroc * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size


    gc.collect() # garbarge collector
    bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, Train_Auroc=epoch_auroc,LR=optimizer.param_groups[0]['lr'])

    return epoch_loss, epoch_auroc

## Inference

In [ ]:
@torch.inference_mode()
def inference(model, dataloader, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, data in bar:
        images = data['image'].to('cuda', dtype=torch.float)
        targets = data['target'].to('cuda', dtype=torch.float)

        batch_size = images.size(0)

        outputs = model(images).squeeze()
        loss = criterion(outputs, targets)

        auroc = binary_auroc(input=outputs.squeeze(), target=targets).item()

        running_loss += (loss.item() * batch_size)
        running_auroc += (auroc * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size

        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, Train_Auroc=epoch_auroc,LR=optimizer.param_groups[0]['lr'])

    gc.collect() # garbarge collector

    return epoch_loss, epoch_auroc

## Run

In [ ]:
def run_training(model, optimizer, scheduler, epoch):
    if torch.cuda.is_available():
        print(f'Using GPU : {torch.cuda.get_device_name()}')

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    history = defaultdict(list)

    for epoch in range(1, epoch + 1):
        gc.collect()

        train_epoch_loss, train_epoch_auroc = model_train(model, optimizer, scheduler,
                                                       dataloader = train_loader,
                                                       epoch=epoch)
        val_epoch_loss, val_epoch_auroc = inference(model, valid_loader, epoch=epoch)

        history['train_loss'].append(train_epoch_loss)
        history['val_loss'].append(val_epoch_loss)
        history['train_auc'].append(train_epoch_auroc)
        history['val_auc'].append(val_epoch_auroc)
        history['lr'].append(scheduler.get_lr()[0])

        # deep copy the model
        if best_epoch_auroc <= val_epoch_auroc:
            print(f"{b_}Validation AUROC Improved ({best_epoch_auroc} ---> {val_epoch_auroc})")
            best_epoch_auroc = val_epoch_auroc
            best_model_wts = copy.deepcopy(model.state_dict())
            model_save_path = 'AUROC{:.4f}_Loss{:.4f}_epoch{:.0f}.bin'.format(val_epoch_auroc, val_epoch_loss, epoch)

            torch.save(model.state_dict(), model_save_path)

            # Save a model file from the current directory
            print(f'Model Saved!{sr_}')

        print()

    end = time.time()
    time_elapsed = end - start

    print(f'Train has completed in {(time_elapsed // 3600):.0f}h:{((time_elapsed % 3600) // 60):.0f}m:{((time_elapsed % 3600) % 60):.0f}s')
    print(f'Best AUROC : {best_epoch_auroc:.4f}')

    # load best model weight
    model.load_state_dict(best_model_wts)

    return model, history


In [ ]:
def prepare_loaders(df, train_batch_size, val_batch_size, fold=0):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    train_dataset = SkinDataSet(df_train, transforms=data_transforms['train'])
    valid_dataset = ISICDataset(df_valid, transforms=data_transforms['valid'])

    train_loader = DataLoader(train_dataset, batch_size=train_batch_size,
                              shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size,
                              shuffle=False, pin_memory=True)

    return train_loader, valid_loader

In [ ]:
# Loader ready!
train_loader, valid_loader= prepare_loaders(df, train_batch_size=32, val_batch_size=64)

lr = 1e-4
weight_decay = 1e-6
optimizer = optim.Adam(model.parameters(), lr, weight_decay=weight_decay)
scheudler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10316, eta_min=1e-6)


model, history = run_training(model, optimizer, scheudler, epoch=50)



Using GPU : Tesla T4


100%|██████████| 26/26 [00:25<00:00,  1.04it/s, Epoch=1, LR=0.0001, Train_Auroc=0.507, Train_Loss=0.521]


Validation AUROC Improved (-inf ---> 0.5070936187921564)
Model Saved!



100%|██████████| 26/26 [00:25<00:00,  1.01it/s, Epoch=2, LR=0.0001, Train_Auroc=0.513, Train_Loss=13.7]


Validation AUROC Improved (0.5070936187921564 ---> 0.5126313881094533)
Model Saved!



...

100%|██████████| 26/26 [00:24<00:00,  1.05it/s, Epoch=47, LR=9.82e-5, Train_Auroc=0.513, Train_Loss=0.471]


100%|██████████| 26/26 [00:25<00:00,  1.03it/s, Epoch=48, LR=9.81e-5, Train_Auroc=0.513, Train_Loss=0.436]


100%|██████████| 26/26 [00:24<00:00,  1.04it/s, Epoch=49, LR=9.8e-5, Train_Auroc=0.513, Train_Loss=0.526]


100%|██████████| 26/26 [00:25<00:00,  1.03it/s, Epoch=50, LR=9.79e-5, Train_Auroc=0.513, Train_Loss=0.481]


Train has completed in 0h:43m:10s
Best AUROC : 0.5142


In [ ]:
torch.save(model, 'model.pth')

In [ ]:
history = pd.DataFrame.from_dict(history)
history.to_csv("history.csv", index=False)